In [1]:
!pip install pyspark
!pip install py4j
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark
from pyspark.context import SparkContext
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col 
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import min, max, mean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 60.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=69a2d0a3a8345ebb01f555a8de7116d3917d718fcc494ce266105e2ae35fe903
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
nba = files.upload()

Saving nba.csv to nba.csv


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
.appName("ALTEN")  \
.config('spark.ui.port','4050') \
.getOrCreate()

In [6]:
spark

In [11]:
df = spark.read.csv('nba.csv')

In [12]:
df.show()


+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|                 _c0|           _c1|   _c2|     _c3| _c4|   _c5|   _c6|              _c7|       _c8|
+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|                Name|          Team|Number|Position| Age|Height|Weight|          College|    Salary|
|       Avery Bradley|Boston Celtics|   0.0|      PG|25.0|   6-2| 180.0|            Texas| 7730337.0|
|         Jae Crowder|Boston Celtics|  99.0|      SF|25.0|   6-6| 235.0|        Marquette| 6796117.0|
|        John Holland|Boston Celtics|  30.0|      SG|27.0|   6-5| 205.0|Boston University|      null|
|         R.J. Hunter|Boston Celtics|  28.0|      SG|22.0|   6-5| 185.0|    Georgia State| 1148640.0|
|       Jonas Jerebko|Boston Celtics|   8.0|      PF|29.0|  6-10| 231.0|             null| 5000000.0|
|        Amir Johnson|Boston Celtics|  90.0|      PF|29.0|   6-9| 240.0|          

In [41]:
df = spark.read.format("csv").option("header","True").load("nba.csv")
df.show()

+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|                Name|          Team|Number|Position| Age|Height|Weight|          College|    Salary|
+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|       Avery Bradley|Boston Celtics|   0.0|      PG|25.0|   6-2| 180.0|            Texas| 7730337.0|
|         Jae Crowder|Boston Celtics|  99.0|      SF|25.0|   6-6| 235.0|        Marquette| 6796117.0|
|        John Holland|Boston Celtics|  30.0|      SG|27.0|   6-5| 205.0|Boston University|      null|
|         R.J. Hunter|Boston Celtics|  28.0|      SG|22.0|   6-5| 185.0|    Georgia State| 1148640.0|
|       Jonas Jerebko|Boston Celtics|   8.0|      PF|29.0|  6-10| 231.0|             null| 5000000.0|
|        Amir Johnson|Boston Celtics|  90.0|      PF|29.0|   6-9| 240.0|             null|12000000.0|
|       Jordan Mickey|Boston Celtics|  55.0|      PF|21.0|   6-8| 235.0|          

In [42]:
df = df.na.drop()
df.show()

+--------------------+--------------+------+--------+----+------+------+--------------+----------+
|                Name|          Team|Number|Position| Age|Height|Weight|       College|    Salary|
+--------------------+--------------+------+--------+----+------+------+--------------+----------+
|       Avery Bradley|Boston Celtics|   0.0|      PG|25.0|   6-2| 180.0|         Texas| 7730337.0|
|         Jae Crowder|Boston Celtics|  99.0|      SF|25.0|   6-6| 235.0|     Marquette| 6796117.0|
|         R.J. Hunter|Boston Celtics|  28.0|      SG|22.0|   6-5| 185.0| Georgia State| 1148640.0|
|       Jordan Mickey|Boston Celtics|  55.0|      PF|21.0|   6-8| 235.0|           LSU| 1170960.0|
|        Kelly Olynyk|Boston Celtics|  41.0|       C|25.0|   7-0| 238.0|       Gonzaga| 2165160.0|
|        Terry Rozier|Boston Celtics|  12.0|      PG|22.0|   6-2| 190.0|    Louisville| 1824360.0|
|        Marcus Smart|Boston Celtics|  36.0|      PG|22.0|   6-4| 220.0|Oklahoma State| 3431040.0|
|     Jare

In [43]:
df = df.filter("Number > 30").filter("Salary > 5000000")
df.show()

+------------------+--------------------+------+--------+----+------+------+----------------+----------+
|              Name|                Team|Number|Position| Age|Height|Weight|         College|    Salary|
+------------------+--------------------+------+--------+----+------+------+----------------+----------+
|       Jae Crowder|      Boston Celtics|  99.0|      SF|25.0|   6-6| 235.0|       Marquette| 6796117.0|
| Patrick Patterson|     Toronto Raptors|  54.0|      PF|27.0|   6-9| 235.0|        Kentucky| 6268675.0|
|     Blake Griffin|Los Angeles Clippers|  32.0|      PF|27.0|  6-10| 251.0|        Oklahoma|18907726.0|
|      Kosta Koufos|    Sacramento Kings|  41.0|       C|27.0|   7-0| 265.0|      Ohio State| 7700000.0|
|     Tobias Harris|     Detroit Pistons|  34.0|      SF|23.0|   6-9| 235.0|       Tennessee|16000000.0|
|      Corey Brewer|     Houston Rockets|  33.0|      SG|30.0|   6-9| 186.0|         Florida| 8229375.0|
|     Zach Randolph|   Memphis Grizzlies|  50.0|      P